# Random forest classification (multi-node, multi-GPU)

<table>
    <tr>
        <td>
            <img src="../_img/dask-horizontal.svg" width="300">
        </td>
        <td>
            <img src="../_img/rapids.svg" width="300">
        </td>
        <td>
            <img src="../_img/snowflake.png" width="300">
        </td>
    </tr>
</table>

This notebook describes a machine learning training workflow using the famous [NYC Taxi Dataset](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page). That dataset contains information on taxi trips in New York City.

In this exercise, you'll use `dask-cudf` to load a subset of the data and `cuml` to answer this classification question:

> based on characteristics that can be known at the beginning of a trip, will this trip result in a high tip?

## Use RAPIDS libraries

RAPIDS is a collection of libraries which enable you to take advantage of NVIDIA GPUs to accelerate machine learning workflows. This exercise uses the following RAPIDS packages:
    
* [`dask-cudf`](https://docs.rapids.ai/api/cudf/stable/dask-cudf.html): distributed `cudf` dataframes using Dask
* [`cuml`](https://github.com/rapidsai/cuml): machine learning training and evaluation, similar to `scikit-learn`

These libraries are already available in the `saturncloud/saturn-gpu` images by default. For more information on RAPIDS, see ["Getting Started"](https://rapids.ai/start.html) in the RAPIDS docs.

<hr>

## Initialize A Dask GPU Cluster

This tutorial uses multiple machines to show how to apply more computing resources to machine learning training. This is done with Dask. Saturn Cloud offers managed Dask clusters, which can be provisioned and modified programmatically.

The code below creates a Dask cluster using [`dask-saturn`](https://github.com/saturncloud/dask-saturn), the official Dask client for Saturn Cloud. It creates a cluster with the following specs:

* `n_workers=3` --> 3 machines in the cluster
* `scheduler_size='medium'` --> the Dask scheduler will have 4GB of RAM and 2 CPU cores
* `worker_size='g4dnxlarge'` --> each worker machine will have 4 CPU cores, 16GB of RAM, 16GB of GPU RAM, and 1 GPU

To see a list of possible sizes, run the code below.

In [1]:
import dask_saturn

dask_saturn.describe_sizes()

{'medium': 'Medium - 2 cores - 4 GB RAM',
 'large': 'Large - 2 cores - 16 GB RAM',
 'xlarge': 'XLarge - 4 cores - 32 GB RAM',
 '2xlarge': '2XLarge - 8 cores - 64 GB RAM',
 '4xlarge': '4XLarge - 16 cores - 128 GB RAM',
 '8xlarge': '8XLarge - 32 cores - 256 GB RAM',
 '12xlarge': '12XLarge - 48 cores - 384 GB RAM',
 '16xlarge': '16XLarge - 64 cores - 512 GB RAM',
 'g4dnxlarge': 'T4-XLarge - 4 cores - 16 GB RAM - 1 GPU',
 'g4dn4xlarge': 'T4-4XLarge - 16 cores - 64 GB RAM - 1 GPU',
 'g4dn8xlarge': 'T4-8XLarge - 32 cores - 128 GB RAM - 1 GPU',
 'p32xlarge': 'V100-2XLarge - 8 cores - 61 GB RAM - 1 GPU',
 'p38xlarge': 'V100-8XLarge - 32 cores - 244 GB RAM - 4 GPU',
 'p316xlarge': 'V100-16XLarge - 64 cores - 488 GB RAM - 8 GPU'}

The `dask-saturn` code below creates two important objects: a cluster and a client.

* `cluster`: knows about and manages the scheduler and workers
    - can be used to create, resize, reconfigure, or destroy those resources
    - knows how to communicate with the scheduler, and where to find logs and diagnostic dashboards
* `client`: tells the cluster to do things
    - can send work to the cluster
    - can restart all the worker processes
    - can send data to the cluster or pull data back from the cluster

In [2]:
import dask
from dask.distributed import Client, wait
from dask_saturn import SaturnCluster

n_workers = 3
cluster = SaturnCluster(
    n_workers=n_workers, 
    scheduler_size="medium", 
    worker_size="g4dnxlarge",
)
client = Client(cluster)
cluster

[2021-01-21 15:44:52] INFO - dask-saturn | Starting cluster. Status: pending
[2021-01-21 15:45:01] INFO - dask-saturn | Starting cluster. Status: pending
[2021-01-21 15:45:13] INFO - dask-saturn | Cluster is ready
[2021-01-21 15:45:13] INFO - dask-saturn | Registering default plugins
[2021-01-21 15:45:13] INFO - dask-saturn | {'tcp://10.0.10.129:43827': {'status': 'OK'}, 'tcp://10.0.21.103:40637': {'status': 'OK'}}


If you created your cluster here in this notebook, it might take a few minutes for all your nodes to become available. You can run the chunk below to block until all nodes are ready.

>**Pro tip**: Create and/or start your cluster in the Saturn UI if you want to get a head start!

In [3]:
cluster.scheduler_info

{'workers': {}}

In [4]:
client

Client Scheduler: tcp://d-aaron-examples-gpu-41446557ab86456d8e5e62b6e21aa4ab.main-namespace:8786 Dashboard: https://d-aaron-examples-gpu-41446557ab86456d8e5e62b6e21aa4ab.community.saturnenterprise.io,Cluster Workers: 2 Cores: 8 Memory: 31.00 GB


In [5]:
client.wait_for_workers(n_workers=n_workers)

### Monitor Resource Usage

This tutorial aims to teach you how to take advantage of multiple GPUs for data science workflows. To prove to yourself that Dask RAPIDS are utilizing the GPUs, it's important to understand how to monitor that utilization while your code is running. If you already know how to do that, skip to the next section.

Print the `cluster` object in a notebook renders a widget that shows the number of workers, available CPU and memory, and a dashboard link.

In [6]:
cluster

Click that dashboard link to view some diagnostic information about the Dask cluster. This can be used to view the current resource utilization of workers in the cluster and lots of information about what they're currently working on.

There are a couple other dashboard pages worth viewing for GPU memory and utilization that are not listed on the navbar. Run the code below to view them.

In [7]:
from IPython.display import display, HTML

gpu_links = f"""
<b>GPU Dashboard links</b>
<ul>
<li><a href="{client.dashboard_link}/individual-gpu-memory" target="_blank">GPU memory</a></li>
<li><a href="{client.dashboard_link}/individual-gpu-utilization" target="_blank">GPU utilization</a></li>
</ul>
"""
display(HTML(gpu_links))

<hr>

## Connect to Snowflake

This example uses data stored in a Snowflake data warehouse that is managed by the team at Saturn Cloud. We've set up a read-only user for use in these examples. If you would like to access data stored in your own Snowflake account, you should set up [Credentials](https://www.saturncloud.io/docs/concepts/credentials/) for your account, user, and password then set the other connection information accordingly. For more details on Snowflake connection information, see ["Connecting to Snowflake"](https://docs.snowflake.com/en/user-guide/python-connector-example.html#connecting-to-snowflake) in the `snowflake-connector-python` docs.

Note that in order to update environment variables your Jupyter server will need to be stopped.

In [8]:
import os
import pandas as pd
import cudf
import dask_cudf as cudd

import snowflake.connector

conn_info = {
    "account": os.environ['EXAMPLE_SNOWFLAKE_ACCOUNT'],
    "user": os.environ['EXAMPLE_SNOWFLAKE_USER'],
    "password": os.environ['EXAMPLE_SNOWFLAKE_PASSWORD'],
    "warehouse": 'saturn_warehouse',
    'database': 'saturn_nyc_taxi',
}
conn = snowflake.connector.connect(**conn_info)

<hr>

## Load data

This example is designed to run quickly with small resources. So let's just load a single month of taxi data for training.

The code below loads the data into a `dask-cudf` data frame. Your code here in this notebook can interact with this data structure as if it was just a regular `cudf` data frame, but it is actually a collection of smaller `cudf` data frames spread across the workers in the Dask cluster.

A `cudf` data frame is similar to a `pandas` data frame, but it lives in GPU memory and most operations on it are done on the GPU.

This example uses Snowflake to handle the hard work of creating new features, then creates a `cudf` data frame with the result.

In [9]:
query = """
SELECT
    pickup_taxizone_id,
    dropoff_taxizone_id,
    passenger_count,
    DIV0(tip_amount, fare_amount) > 0.2 AS high_tip,
    DAYOFWEEKISO(pickup_datetime) - 1 AS pickup_weekday,
    WEEKOFYEAR(pickup_datetime) AS pickup_weekofyear,
    HOUR(pickup_datetime) AS pickup_hour,
    (pickup_weekday * 24) + pickup_hour AS pickup_week_hour,
    MINUTE(pickup_datetime) AS pickup_minute
FROM taxi_yellow
WHERE
    DATE(pickup_datetime) = %s
"""


@dask.delayed
def load(conn_info, query, day):
    with snowflake.connector.connect(**conn_info) as conn:
        taxi = conn.cursor().execute(query, day).fetch_pandas_all()
        taxi.columns = taxi.columns.str.lower()
        taxi = cudf.from_pandas(taxi)
        return taxi

In [10]:
def get_dates(start, end):
    date_query = """
    SELECT
        DISTINCT(DATE(pickup_datetime)) as date
    FROM taxi_yellow
    WHERE
        pickup_datetime BETWEEN %s and %s
    """
    dates_df = conn.cursor().execute(date_query, (start, end))
    dates_df = conn.cursor().execute(date_query, ("2019-01-01", "2019-01-31"))
    return dates_df.fetch_pandas_all()['DATE'].tolist()


dates = get_dates("2019-01-01", "2019-01-31")

In [11]:
taxi = cudd.from_delayed([load(conn_info, query, day) for day in dates])

In [12]:
taxi

,pickup_taxizone_id,dropoff_taxizone_id,passenger_count,high_tip,pickup_weekday,pickup_weekofyear,pickup_hour,pickup_week_hour,pickup_minute
npartitions=31,,,,,,,,,
,int16,int16,int16,bool,int8,int8,int8,int16,int8
,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...


Dask performs computations in a [lazy manner](https://tutorial.dask.org/01x_lazy.html), so we persist the dataframe to perform data loading and feature processing and load into GPU memory.

In [13]:
numeric_feat = [
    "pickup_weekday",
    "pickup_weekofyear",
    "pickup_hour",
    "pickup_week_hour",
    "pickup_minute",
    "passenger_count",
]
categorical_feat = [
    "pickup_taxizone_id",
    "dropoff_taxizone_id",
]
features = numeric_feat + categorical_feat
y_col = "high_tip"

taxi_train = taxi[features + [y_col]]
taxi_train[features] = taxi_train[features].astype("float32").fillna(-1)
taxi_train[y_col] = taxi_train[y_col].astype("int32").fillna(-1)

In [14]:
%%time

taxi_train = taxi_train.persist()
_ = wait(taxi_train)

CPU times: user 158 ms, sys: 1.71 ms, total: 159 ms
Wall time: 17.8 s


The code below computes the size of this dataset in memory.

In [15]:
print(
    f"Num rows: {len(taxi_train)}, Size: {taxi_train.memory_usage(deep=True).sum().compute() / 1e6} MB"
)

Num rows: 7667648, Size: 276.035328 MB


You can examine the structure of the data with `cudf` commands:

`.head()` = view the first few rows

In [16]:
taxi_train.head()

,pickup_weekday,pickup_weekofyear,pickup_hour,pickup_week_hour,pickup_minute,passenger_count,pickup_taxizone_id,dropoff_taxizone_id,high_tip
0,5.0,4.0,19.0,139.0,30.0,1.0,255.0,263.0,0
1,5.0,4.0,23.0,143.0,58.0,6.0,164.0,230.0,1
2,5.0,4.0,23.0,143.0,56.0,1.0,249.0,33.0,1
3,5.0,4.0,23.0,143.0,59.0,2.0,264.0,264.0,0
4,5.0,4.0,23.0,143.0,59.0,1.0,141.0,151.0,0


`.dtypes` = list all the columns and the type of data in them

In [17]:
taxi.dtypes

pickup_taxizone_id     int16
dropoff_taxizone_id    int16
passenger_count        int16
high_tip                bool
pickup_weekday          int8
pickup_weekofyear       int8
pickup_hour             int8
pickup_week_hour       int16
pickup_minute           int8
dtype: object

When we say that a `dask-cudf` dataframe is a *distributed* data frame, that means that it comprises multiple smaller `cudf` data frames. Run the following to see how many of these pieces (called "partitions") there are.

In [18]:
taxi.npartitions

31

`taxi_train` is a `dask_cudf` dataframe that will be passed in to a machine learning model. Since this is a binary classification task, before proceeding we should examine the distributions of 1s and 0s in the target. This can be done with by combining `.groupby()` and `.count()`

<details><summary>(click here to learn why data scientists do this)</summary>

**Target Distribution**
    
In binary classification tasks, we ask machine learning models to learn the difference between records that produced a 0 for some target and those that produce a 1. In this example:
    
* `high_tip = 1`: "this trip resulted in a high tip"
* `high_tip = 0`: "this trip did not result in a high tip"
    
The "distribution" of this target just means "what % of the trips ended in a high tip?". This is important to understand because we might have to change our modelling approach if the distribution was uneven. For example, if 99% of trips did NOT end in a high tip, a model that just always guessed "not a high tip" would achieve an accuracy of 99%.

</details>

In [19]:
taxi_train.groupby("high_tip")["high_tip"].count().compute()

high_tip
1    4001846
0    3665802
Name: high_tip, dtype: int64

<hr>

## Train a Model

Now that the data have been prepped, it's time to build a model!

For this task, we'll use the `RandomForestClassifier` from `cuml`. If you've never used a random forest or need a refresher, consult ["Forests of randomized trees"](https://scikit-learn.org/stable/modules/ensemble.html#forest) in the `sciki-learn` documentation.

The code below initializes a random forest classifier with the following parameter values.

* `n_estimators=100` = create a 100-tree forest
* `max_depth=10` = stop growing a tree once it contains a leaf node that is 10 levels below the root

All other parameters use the defaults from `RandomForestClassifier`.

<details><summary>(click here to learn why data scientists do this)</summary>

**Setting max_depth**

Tree-based models split the training data into smaller and smaller groups, to try to group together records with similar values of the target. A tree can be thought of as a collection of rules like `pickup_hour greater than 11` and `pickup_minute less than 31.0`. As you add more rules, those groups (called "leaf nodes") get smaller. In an extreme example, a model could create a tree with enough rules to place each record in the training data into its own group. That would probably take a lot of rules, and would be referred to as a "deep" tree.

Deep trees are problematic because their descriptions of the world are too specific to be useful on new data. Imagine training a classification model to predict whether or not visitors to a theme park will ride a particular rollercoaster. You could measure the time down to the millisecond that every guest's ticket is scanned at the entrance, and a model might learn a rule like *"if the guest has been to the park before and if the guest is older than 40 and younger than 41, and if the guest is staying at Hotel A and if the guest enters the park after 1:00:17.456 and if the guest enters the park earlier than 1:00:17.995, they will ride the rollercoaster"*. This is very very unlikely to ever match any future visitors, and if it does it's unlikely that this prediction will be very good unless you have some reason to believe that a visitor arriving at 1:00:18 instead of 1:00:17 really changes the probability that they'll ride that rollercoaster.

To prevent this situation (called "overfitting"), most tree-based machine learning algorithms accept parameters that control how deep the trees can get. `max_depth` is common, and says "don't create a rule more complex than this". In the example above, that rule has a depth of 7.

1. visiting the park
2. has been to the park before?
3. older than 40?
4. younger than 41?
5. staying at Hotel A?
6. entered the park after 1:00:17.456?
7. entered the park before 1:00:17.995?

Setting `max_depth = 5` would have prevented those weirdly-specific timing rules from ever being generated.

Choosing good values for this parameter is part art, part science, and is outside the scope of this tutorial.

</details>

In [20]:
from cuml.dask.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(n_estimators=100, max_depth=10, seed=42)

With the classifier created, fit it to some data! The code below uses `%%time` to print out a timing, so you can see how long it takes to train. This can be used to compare `cuml` to methods explored in other notebooks, or to test how changing some parameters to `RandomForestClassifier` changes the runtime for training.

You might notice that the code below looks identical to the code from the non-Dask, single node example covered in the [single-node RAPIDS example](./rf-rapids.ipynb). This is a really nice feature of `cuml`. `cuml` models can accept training data in `dask_cudf` or `cudf` format. Pretty cool, right?

As soon as you run the cell below, switch over to the tab where you started up the Dask diagnostic dashboard(s). You should see some work being done! If it finishes too quickly, change `n_estimators` in the cell above to a larger value, like 1000.

In [21]:
%%time
_ = rfc.fit(taxi_train[features], taxi_train[y_col])

CPU times: user 236 ms, sys: 157 ms, total: 393 ms
Wall time: 5.84 s


## Save Model

The distributed model objects from `cuml` cannot be pickled directly. Some of the model objects come with methods for generating a single-node equivalent (see ["Pickling cuML Models"](https://docs.rapids.ai/api/cuml/stable/pickling_cuml_models.html#Distributed-Model-Pickling)), but unfortunately `cuml.dask.ensemble.RandomForestClassifier` is not one of those.

Your best option for storing this model is to use a private method, `._concat_treelite_models()`. This produces a regular `cuml.ensemble.randomforestclassifier.RandomForestClassifier`, which you can use for scoring on local data.

In [22]:
local_model = rfc._concat_treelite_models()

You can use this model to score on local data, like in a `pandas` data frame or `numpy` array.

In [23]:
local_model.predict_proba(taxi_train.head(100)[features])

,0,1
0,0.05,0.95
1,0.24,0.76
2,0.15,0.85
3,0.08,0.92
4,0.13,0.87
...,...,...
95,0.32,0.68
96,0.15,0.85
97,0.20,0.80
98,0.15,0.85


This model can also be saved for later use. There are several ways to do this, but `cloudpickle` is likely to give you the best experience. It handles some common drawbacks of the built-in `pickle` library.

`cloudpickle` can be used to write a Python object to bytes, and to create a Python object from that binary representation.

In [24]:
import cloudpickle
import os

MODEL_PATH = "models"
if not os.path.exists(MODEL_PATH):
    os.makedirs(MODEL_PATH)

with open(f"{MODEL_PATH}/random_forest_dask_rapids.pkl", "wb") as f:
    cloudpickle.dump(local_model, f)

<hr>

## Calculate metrics on test set

Machine learning training tries to create a model which can produce useful results on new data that it didn't see during training. To test how well we've accomplished that in this example, read in another month of taxi data.

In [25]:
test_dates = get_dates("2019-02-01", "2019-02-28")
taxi_test = cudd.from_delayed([load(conn_info, query, day) for day in test_dates])

In [26]:
taxi_test = taxi_test[features + [y_col]]
taxi_test[features] = taxi_test[features].astype("float32").fillna(-1)
taxi_test[y_col] = taxi_test[y_col].astype("int32").fillna(-1)

In [27]:
taxi_test = taxi_test.persist()
_ = wait(taxi_test)

`cuml` comes with many functions for calculating metrics that describe how well a model's predictions match the actual values. For a complete list, see [the cuml API docs](https://docs.rapids.ai/api/cuml/stable/api.html#metrics-regression-classification-and-distance) or run the code below.

In [28]:
import cuml.metrics

[m for m in dir(cuml.metrics) if not m.startswith("_")]

['PAIRWISE_DISTANCE_METRICS',
 'accuracy',
 'accuracy_score',
 'adjusted_rand_score',
 'cluster',
 'completeness_score',
 'confusion_matrix',
 'entropy',
 'homogeneity_score',
 'log_loss',
 'mean_absolute_error',
 'mean_squared_error',
 'mean_squared_log_error',
 'mutual_info_score',
 'pairwise_distances',
 'precision_recall_curve',
 'r2_score',
 'regression',
 'roc_auc_score',
 'trustworthiness',
 'utils']

This tutorial uses the `roc_auc_score` to evaluate the model. This metric measures the area under the [receiver operating characteristic](https://en.wikipedia.org/wiki/Receiver_operating_characteristic) curve. Values closer to 1.0 are desirable.

In [29]:
from cuml.metrics import roc_auc_score

preds = rfc.predict_proba(taxi_test[features])[1]

As of this writing, `cuml.metrics.roc_auc_score` does not support Dask collections as inputs. The code below uses `.compute()` to create numpy arrays instead. This isn't ideal because it means that the amount of data you can score on is limited by the size of the instance you're running this code on. Hopefully support for Dask collections will be added in a future verison of `cuml`.

In [30]:
roc_auc_score(taxi_test[y_col].compute(), preds.compute())

0.5473746657371521

<hr>

## Next Steps

In this tutorial, you learned how to train a model for a binary classification task, using `cuml` and Dask. Training took around 3 seconds for a dataset that was 0.31 GB in memory, a huge improvement on the almost 9 minutes it took to [train the same model using `scikit-learn`](./01-rf-scikit.ipynb)!

You've reached the end of this set of exercises on using Snowflake + RAPIDS + Dask to accomplish distributed machine learning training using GPUs.

If you want to try working with larger datasets, change the code in this notebook to use more data, or explore how to programmatically resize your Dask cluster by following the tips in ["Managing Dask Resources for Machine Learning Training"](https://www.saturncloud.io/docs/tips-and-tricks/training-on-dask).

<hr>